# Data Extraction from SQL database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Data Overview

In [2]:
%%sql
SELECT *
    FROM sqlite_master
   WHERE type ='table';
    

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
    FROM facts
   LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Data Exploration and Analysis
### Summary Statistics

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


>Note that there is a country with a population of Zero and another with a population of 7.3 billion people, which is larger than the population of the entire planet.

In [5]:
%%sql 
SELECT name
    FROM facts
  WHERE population = (SELECT MIN(population) FROM facts)

Done.


name
Antarctica


In [6]:
%%sql 
SELECT name
    FROM facts
  WHERE population = (SELECT MAX(population) FROM facts)

Done.


name
World


### Exploring Average Population and Area


In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
 WHERE name != 'World';


Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [8]:
%%sql
SELECT AVG(population), AVG(area)
    FROM facts;

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


### Exploring Densely Populated Countries

In [9]:
%%sql 
SELECT name 
    FROM facts
  WHERE (population > (SELECT AVG(population) FROM facts)) 
  AND area < (SELECT AVG(area) FROM facts)

Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


### Exploring Countries with the Largest Population and Growth Rate

In [10]:
%%sql 
SELECT name, population, population_growth
 FROM facts
ORDER BY population DESC
LIMIT 5;
    

Done.


name,population,population_growth
World,7256490011,1.08
China,1367485388,0.45
India,1251695584,1.22
European Union,513949445,0.25
United States,321368864,0.78


In [11]:
%%sql 
SELECT name, population, population_growth
 FROM facts
ORDER BY population_growth DESC
LIMIT 5;

Done.


name,population,population_growth
South Sudan,12042910,4.02
Malawi,17964697,3.32
Burundi,10742276,3.28
Niger,18045729,3.25
Uganda,37101745,3.24


### Top 5 Countries with the Highest Ratios of Water to Land

In [12]:
%%sql 
SELECT name, CAST(area_water AS Float)/CAST(area_land AS Float) AS ratio_water_land, area_water, area_land
 FROM facts
ORDER BY Ratio_water_land DESC
LIMIT 5;

Done.


name,ratio_water_land,area_water,area_land
British Indian Ocean Territory,905.6666666666666,54340,60
Virgin Islands,4.520231213872832,1564,346
Puerto Rico,0.5547914317925592,4921,8870
"Bahamas, The",0.3866133866133866,3870,10010
Guinea-Bissau,0.2846728307254623,8005,28120


### Top 5 Countries with the Higher Death Rate than Birth Rate

In [13]:
%%sql 
SELECT name, birth_rate, death_rate, 
       CAST(death_rate AS Float)/CAST(birth_rate AS Float) AS death_birth_ratio
 FROM facts
ORDER BY death_birth_ratio DESC
LIMIT 5;

Done.


name,birth_rate,death_rate,death_birth_ratio
Bulgaria,8.92,14.44,1.6188340807174888
Serbia,9.08,13.66,1.5044052863436124
Latvia,10.0,14.31,1.431
Lithuania,10.1,14.27,1.4128712871287128
Hungary,9.16,12.73,1.3897379912663756
